# Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import time

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,rand_score



# Functions

In [2]:
import numpy as np

def minkowski_distance(x:np.array,y:np.array,p:int) -> float:
    """
    Função que implementa a distância de minkowski entre dois pontos x e y.
    Recebe os pontos e um parâmetro p que controla a distância.
    """
    diff_power = np.power(np.abs(x - y),p)
    return np.power(np.sum(diff_power),1/p) 

def pairwise_minkowski_distance(X:np.array,p:int) -> np.array:
    """
    Função que implementa a distância de minkowski entre os pontos em uma matriz de pontos X.
    Recebe a matriz e um parâmetro p.
    """
    diff_power = np.power(np.abs(X[:,None,:] - X[None,:,:]),p)
    return np.power(np.sum(diff_power,axis=-1),1/p)

def cluster_distance(x_i:np.array,clusters:list,p:int) -> float:
    """
    Função que calcula a distância de um ponto a um determinado cluster, definida como a menor distância dentre as distâncias do ponto para todos os pontos no cluster.
    Recebe o ponto, o cluster e o parâmetro p da distância.
    """
    cluster_distances = np.array([minkowski_distance(x_i,c_i,p) for c_i in clusters])
    return cluster_distances

def compute_cluster_radius(points:np.array,clusters:list,p:int) -> float:
    """
    Função que computa o raio máximo de um cluster.
    Recebe um conjunto de pontos, o cluster e o parâmetro p da distância.
    """
    radiuses = np.zeros(len(points))
    for idx,point in enumerate(points):
        radiuses[idx] = np.min(cluster_distance(point,clusters,p))
    return np.max(radiuses) 
        
def approx_k_clustering(points:np.array,num_clusters:int,p:int,seed:int) -> tuple:
    """
    Função que implementa o algoritmo 2-aproximado para o problema dos k-centros.
    Recebe um conjunto de pontos, o número k de clusters, o parâmetro p da distância e uma semente aleatória.
    """
    if num_clusters > len(points):
        return points,np.array([*range(len(points))])
    
    np.random.seed(seed)
    rand_start = points[np.random.randint(0,len(points))]
    clusters = [rand_start]

    # Computa os clusters
    while len(clusters) < num_clusters:
        radiuses = np.zeros(len(points))

        for idx,point in enumerate(points):
            radiuses[idx] = np.min(cluster_distance(point,clusters,p))

        max_rad_idx = np.argmax(radiuses)
        tgt_point = points[max_rad_idx,:]

        clusters.append(tgt_point)

    # Classifica os pontos
    point_classes = np.zeros(len(points))

    for idx,point in enumerate(points):
        cluster_dists = cluster_distance(point,clusters,p)
        point_classes[idx] = np.argmin(cluster_dists)
    
    return clusters,point_classes

# Datasets

In [3]:
datasets = []

## Mammographic Mass
Link:https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass

Tarefa: Prever se o tumor mamário é maligno ou benigno (Variável categórica binária)

In [4]:
df = pd.read_csv("./datasets/mammographic_masses.data",delimiter=',',header=None)

idx_to_keep = []
for idx,i in enumerate(df.to_numpy()):
    if len(np.where(i == '?')[0]) == 0:
        idx_to_keep.append(idx)

df_numpy = df.iloc[idx_to_keep].to_numpy(dtype=int)

true_classes = df_numpy[:,-1]
points = df_numpy[:,:5]
num_classes = 2

datasets.append(("MM",points,true_classes,num_classes))
print(points.shape,true_classes.shape)
df.head()


(830, 5) (830,)


,0,1,2,3,4,5
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1


## South German Credit
Link: https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29

Tarefa: Classificar o indivíduo de acordo com seu histórico de crédito (Variável categórica "moral" com 5 valores possíveis)

In [5]:
df = pd.read_csv("./datasets/SouthGermanCredit.asc",delimiter = ' ')

true_classes = df['moral'].to_numpy()
df_cluster = df.drop(columns=['moral'])

points = df_cluster.to_numpy()
num_classes = 5

datasets.append(("SGC",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(1000, 20) (1000,)


,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


## Blood Transfusion Service Center
Link: https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

Tarefa: Prever a variável categórica binária que indica se a pessoa doou sangue em março de 2007

In [6]:
df = pd.read_csv("./datasets/transfusion.data",delimiter=',')

df_numpy = df.to_numpy()
true_classes = df_numpy[:,-1]
points = df_numpy[:,:-1]
num_classes = 2

datasets.append(("BT",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(748, 4) (748,)


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


## Audit Data
Link: https://archive.ics.uci.edu/ml/datasets/Audit+Data

Tarefa: Prever a variável categórica binária "risk"

In [7]:
df = pd.read_csv("./datasets/audit_risk.csv")
df = df.drop(columns=['LOCATION_ID']) # Identificador que não será utilizado na clusterização
df = df.dropna()

true_classes = df.iloc[:,-1].to_numpy(int)
points = df.iloc[:,:-1].to_numpy(float)
num_classes = 2

datasets.append(("AD",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(775, 25) (775,)


,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,0.4,0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1
1,3.89,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0,0.2,0.0,2.0,1.416,0.4,0.5,0.2832,0


## Drug Consumption
Link: https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Tarefa: Prever uma classe que identifica o número de drogas já utilizadas pelo indivíduo (18 possíveis)

In [8]:
df = pd.read_csv("./datasets/drug_consumption.data",header=None).iloc[:,1:]

for category in ["CL" + str(i) for i in range(7)]:
    if category == "CL0":
        df = df.replace(category,0)
    else:
        df = df.replace(category,1)

points = df.iloc[:,:12].to_numpy(dtype=float)

true_classes = df.iloc[:,12:].sum(axis=1).to_numpy(dtype=int)
num_classes = 18

datasets.append(("DC",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(1885, 12) (1885,)


,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,0,0,0,0,0,0,0,1,0,0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,1,0,1,0,1,1,0,1,0,0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,0,0,0,0,0,0,1,0,0,0
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,0,0,1,0,0,0,0,1,0,0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,1,0,0,1,0,0,1,1,0,0


## Myocardial infarction complications
Link: https://archive.ics.uci.edu/ml/datasets/Myocardial+infarction+complications

Tarefa: Prever a variável categórica "Lethal Outcome", que pode assumir 8 valores que representam a causa da morte do paciente.

In [9]:
df = pd.read_csv("./datasets/MI.data",header=None).iloc[:,1:]
df = df.replace('?',0)


points = df.iloc[:,:110].to_numpy(float)
      
true_classes = df.iloc[:,-1].to_numpy(int)
num_classes = 8

datasets.append(("MI",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(1700, 110) (1700,)


,1,2,3,4,5,6,7,8,9,10,...,114,115,116,117,118,119,120,121,122,123
0,77,1,2,1,1,2,0,3,0,7,...,0,0,0,0,0,0,0,0,0,0
1,55,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,52,1,0,0,0,2,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
3,68,0,0,0,0,2,0,2,0,3,...,0,0,0,0,0,0,1,0,0,0
4,60,1,0,0,0,2,0,3,0,7,...,0,0,0,0,0,0,0,0,0,0


## Cervical cancer (Risk Factors)
Link: https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29

Tarefa: Prever o resultado binário da biopsia do paciente

In [10]:
df = pd.read_csv("./datasets/risk_factors_cervical_cancer.csv")
df = df.drop(columns = ['STDs: Time since first diagnosis','STDs: Time since last diagnosis']) # Muitos valores faltantes


# Identifica quais linhas não possuem valores faltantes
non_missing_idx = []
for idx,i in enumerate(df.to_numpy()):
    if len(np.where(i == '?')[0]) == 0:
        non_missing_idx.append(idx)

# Substitui os valores faltantes de cada coluna w.r.t. média daquela coluna
for col_idx,col in enumerate(df.columns):
    df[col] = df[col].replace("?",df.iloc[non_missing_idx,col_idx].astype(float).mean())


points = df.iloc[:,:32].to_numpy(float)
true_classes = df['Biopsy'].to_numpy(int)
num_classes = 2
datasets.append(("CC",points,true_classes,num_classes))
print(points.shape,true_classes.shape)

df.head()

(858, 32) (858,)


,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
2,34,1.0,17.142216,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0


## Room Occupancy Estimation
Link: https://archive.ics.uci.edu/ml/datasets/Room+Occupancy+Estimation

Tarefa: Classificar a sala de acordo com o número de moradores

In [11]:
df = pd.read_csv("./datasets/Occupancy_Estimation.csv").iloc[:,2:].sample(3000) # Amostra de tamanho 3000 pois amostras maiores estouram a memória (pairwise distance é uma matriz grande)

true_classes = df.iloc[:,-1].to_numpy(int)
points = df.iloc[:,:-1].to_numpy(float)
num_classes = 4

datasets.append(("ROE",points,true_classes,num_classes))
print(points.shape,true_classes.shape)
df.head()

(3000, 16) (3000,)


,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
1143,25.69,25.75,25.38,25.94,0,0,0,0,0.07,0.05,0.05,0.06,670,-2.215385,0,0,0
6634,25.44,25.44,25.13,25.94,0,0,0,0,0.07,0.05,0.06,0.09,355,0.076923,0,0,0
4100,25.50,25.50,25.13,25.69,0,0,0,0,0.07,0.05,0.06,0.06,385,-0.146154,0,0,0
2704,25.13,25.13,24.81,25.69,10,12,51,35,0.08,0.05,0.06,0.05,360,0.142308,0,0,0
3983,25.69,25.63,25.31,25.81,0,0,0,0,0.07,0.05,0.06,0.06,550,-2.057692,0,0,0


## Wine Quality (Red)
Link: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

Tarefa: Prever a qualidade do vinho tinto (Variável categórica de 0 a 10)

In [12]:
df = pd.read_csv("./datasets/winequality-red.csv",delimiter=';')

true_classes = df['quality'].to_numpy(int)
points = df.iloc[:,:-1].to_numpy(float)

num_classes = 11

datasets.append(("WQ",points,true_classes,num_classes))
print(points.shape,true_classes.shape)
df.head()

(1599, 11) (1599,)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Spambase
Link: https://archive.ics.uci.edu/ml/datasets/Spambase

Tarefa: Prever se o email é ou não spam.

In [13]:
df = pd.read_csv("./datasets/spambase.data",header=None).sample(3000)

true_classes = df.iloc[:,-1].to_numpy(int)
points = df.iloc[:,:-1].to_numpy(float)

num_classes = 2

datasets.append(("SB",points,true_classes,num_classes))
print(points.shape,true_classes.shape)
df.head()

(3000, 57) (3000,)


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
3117,0.00,14.28,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.000,0.000,0.0,0.000,0.000,0.0,1.800,5,9,0
4324,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.000,0.000,0.0,0.000,0.000,0.0,1.000,1,10,0
2292,0.18,0.14,0.25,0.0,0.00,0.0,0.0,0.07,0.0,0.14,...,0.019,0.414,0.0,0.004,0.000,0.0,2.393,40,1795,0
2164,0.07,0.07,0.07,0.0,0.14,0.0,0.0,0.43,0.0,0.00,...,0.056,0.094,0.0,0.000,0.028,0.0,2.394,24,881,0
2510,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.000,0.000,0.0,0.000,0.000,0.0,1.000,1,5,0


# Tests

In [14]:
ilocs = []
for (dataset_name,points,true_classes,num_classes) in datasets:
    print(f"Dataset: {dataset_name}")
    print(f"Points shape:{points.shape}")
    print(f"Number of clusters(classes):{num_classes}")
    
    model_rand_scores = []
    sklearn_rand_scores = []
    
    model_silhouette_scores = []
    sklearn_silhouette_scores = []
    
    model_cluster_radiuses = []
    sklearn_cluster_radiuses = []
    
    model_runtimes = []
    sklearn_runtimes = []
    
    for p in [1,2]:
        pairwise_dist = pairwise_minkowski_distance(points,p)
        for seed in tqdm([*range(1,31)]):
            start = time.time()
            model_clusters,point_classes = approx_k_clustering(points,num_classes,p,seed)
            model_runtimes.append(time.time() - start)
            
            model_rand_scores.append(rand_score(true_classes,point_classes))
            model_silhouette_scores.append(silhouette_score(pairwise_dist,point_classes))
            model_cluster_radiuses.append(compute_cluster_radius(points,model_clusters,p))
            
            kmeans = KMeans(num_classes,random_state=seed)
            start = time.time()
            kmeans.fit(points)
            sklearn_runtimes.append(time.time() - start)

            sklearn_rand_scores.append(rand_score(true_classes,kmeans.labels_))
            sklearn_silhouette_scores.append(silhouette_score(pairwise_dist,kmeans.labels_))
            sklearn_cluster_radiuses.append(compute_cluster_radius(points,[center for center in kmeans.cluster_centers_],p))

        curr_iloc = [dataset_name,num_classes,len(true_classes),points.shape[1],p,np.mean(model_rand_scores),np.std(model_rand_scores),np.mean(model_silhouette_scores),np.std(model_silhouette_scores),
                    np.mean(model_cluster_radiuses),np.std(model_cluster_radiuses),np.mean(model_runtimes),np.std(model_runtimes),
                     np.mean(sklearn_rand_scores),np.std(sklearn_rand_scores),np.mean(sklearn_silhouette_scores),
                     np.std(sklearn_silhouette_scores),np.mean(sklearn_cluster_radiuses),np.std(sklearn_cluster_radiuses),np.mean(sklearn_runtimes),np.std(sklearn_runtimes)]
        ilocs.append(curr_iloc)
    

    




Dataset: MM
Points shape:(830, 5)
Number of clusters(classes):2


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Dataset: SGC
Points shape:(1000, 20)
Number of clusters(classes):5


  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Dataset: SB
Points shape:(3000, 57)
Number of clusters(classes):2


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [15]:
table_cols = ['Dataset','K','No. Instances','No. Dimensions','Distance Factor (P)',
              'Model: Rand Score Mean','Model: Rand Score Std','Model: Silhouette Mean','Model: Silhouette Std',
              'Model: Cluster Radius Mean','Model: Cluster Radius Std','Model: Runtime Mean','Model: Runtime Std',
              'KMeans: Rand Score Mean','KMeans: Rand Score Std','KMeans: Silhouette Mean','KMeans: Silhouette Std',
              'KMeans: Cluster Radius Mean','KMeans: Cluster Radius Std','KMeans: Runtime Mean','KMeans: Runtime Std']


final_table = pd.DataFrame(ilocs,columns=table_cols)

final_table

,Dataset,K,No. Instances,No. Dimensions,Distance Factor (P),Model: Rand Score Mean,Model: Rand Score Std,Model: Silhouette Mean,Model: Silhouette Std,Model: Cluster Radius Mean,...,Model: Runtime Mean,Model: Runtime Std,KMeans: Rand Score Mean,KMeans: Rand Score Std,KMeans: Silhouette Mean,KMeans: Silhouette Std,KMeans: Cluster Radius Mean,KMeans: Cluster Radius Std,KMeans: Runtime Mean,KMeans: Runtime Std
0,MM,2,830,5,1,0.501609,9.281097e-03,0.666486,1.133211e-01,56.666667,...,0.033767,0.001054,0.568332,1.110223e-16,0.474071,1.110223e-16,57.824561,1.421085e-14,0.019634,0.001559
1,MM,2,830,5,2,0.508902,1.996074e-02,0.615823,1.218778e-01,52.924638,...,0.034583,0.001605,0.568332,0.000000e+00,0.488162,1.409092e-02,54.399396,3.425165e+00,0.020601,0.004152
2,SGC,5,1000,20,1,0.515429,1.882557e-02,0.595973,4.620146e-02,2459.466667,...,0.172523,0.006163,0.556397,2.364129e-03,0.560840,1.563608e-03,5281.614646,1.193210e+02,0.045989,0.006017
3,SGC,5,1000,20,2,0.515422,1.883192e-02,0.596145,4.613312e-02,2438.987040,...,0.170417,0.005720,0.556397,2.364129e-03,0.561043,1.563952e-03,5266.182495,1.203989e+02,0.044953,0.005408
4,BT,2,748,4,1,0.643271,7.160691e-04,0.865443,3.371410e-03,5594.033333,...,0.031333,0.002891,0.599156,1.110223e-16,0.685933,0.000000e+00,8627.529915,1.818989e-12,0.016267,0.002048
5,BT,2,748,4,2,0.643271,7.160691e-04,0.866078,3.417054e-03,5567.916217,...,0.030967,0.002303,0.599156,1.110223e-16,0.687255,1.321497e-03,8590.545487,3.698443e+01,0.016484,0.003452
6,AD,2,775,25,1,0.522601,1.110223e-16,0.972046,3.330669e-16,2548.874116,...,0.034900,0.003515,0.524881,0.000000e+00,0.955682,1.110223e-16,4437.524400,9.094947e-13,0.016501,0.008032
7,AD,2,775,25,2,0.522601,0.000000e+00,0.971460,5.858051e-04,1886.943248,...,0.035050,0.002848,0.524881,0.000000e+00,0.958101,2.419227e-03,3117.433588,1.320091e+03,0.015417,0.005894
8,DC,18,1885,12,1,0.817565,4.863508e-02,-0.019302,1.453874e-02,11.579739,...,3.145914,0.058652,0.885220,4.073964e-04,0.042553,3.500742e-03,11.447889,3.699098e-01,0.213198,0.015109
9,DC,18,1885,12,2,0.805866,5.702706e-02,-0.013336,1.427285e-02,8.037531,...,3.197126,0.070073,0.885220,4.073964e-04,0.047181,5.607809e-03,7.816764,3.641250e+00,0.213926,0.016633


In [16]:
final_table.to_csv("Resultados_Experimentos.csv")